In [1]:
# import relevant modules and libraries
import pandas as pd
import numpy as np
import re

Data to be cleaned is real estate data of property for sale in Lagos. The data and its features described appropriately in the 'data_dictionary.pdf' file found in the files folder in the master branch of the project repository. The data is in csv format. Get started by loading the data using the pandas' 'read_csv' method.

In [53]:
# load data
lag = pd.read_csv('lag_houses.csv')

Once dataset has been loaded it is best to take a high level view of the data, this can be done by taking a look at its dimensions using pandas' 'shape' attribute. After this a look at the first few rows using the 'head' method to have a look at some contents of the dataset.

In [54]:
# peek at data
print(f'Shape:{lag.shape}')
lag.head()

Shape:(24773, 10)


,Description,Title,Location,Beds,Baths,Toilets,Is_new,Is_furnished,Is_serviced,Price
0,4 Bedroom Semi Detached Duplex,4 BEDROOM HOUSE FOR SALE,Chevron Lekki Lagos,4.0,5.0,5.0,1,1,1,85000000
1,5 Bedroom Fully Detached Duplex,5 BEDROOM HOUSE FOR SALE,2nd Toll Gate Oral Estate Lekki Lagos,5.0,6.0,6.0,1,1,0,160000000
2,4 Bedroom Semi Detached Duplex+ Jacuzzi,4 BEDROOM HOUSE FOR SALE,2nd Toll Gate Lekki Lagos,4.0,5.0,5.0,1,1,0,68000000
3,5 Bedroom Duplex,5 BEDROOM HOUSE FOR SALE,Ikate Lekki Lagos,5.0,6.0,6.0,1,0,0,290000000
4,3 Bedroom Apartment,3 BEDROOM HOUSE FOR SALE,Lekki Lagos,3.0,4.0,4.0,1,1,0,150000000


As seen above the data set has 24,773 rows and 10 columns. From this view we can infer what types of data are present in the dataset. To be certain about the data types we take a look at them we use pandas' 'info' method. The dtypes attribute can also be used here. 

In [159]:
# try to undertand data types
lag.info()
# lag.dtypes

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19074 entries, 0 to 24772
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Beds           19074 non-null  float64
 1   Baths          19074 non-null  float64
 2   Toilets        19074 non-null  float64
 3   Is_new         19074 non-null  object 
 4   Is_furnished   19074 non-null  object 
 5   Is_serviced    19074 non-null  object 
 6   Price          19074 non-null  float64
 7   Property_type  19074 non-null  object 
 8   Location       19074 non-null  object 
dtypes: float64(4), object(5)
memory usage: 1.5+ MB


Most columns have data types assigned correctly but the Price column notably has its type as object, this means that the contents of that column are not all numbers. To correct this we take a closer look at the price column to assess what impurities are present.

In [56]:
# take closer look at price column
lag['Price'][lag['Price'].str.contains(r'[^\d.]', regex=True, na=False)]

105             $440000
134      130000000/year
963            $1100000
1024           $1500000
1112     300000000/year
              ...      
23869          $1400000
23897          $5000000
23899       1500000/sqm
24338          $2000000
24484          $1500000
Name: Price, Length: 408, dtype: object

As the output above shows there are over 400 dirty entries in the  price column. Some prices are designated in dollars and some prices appear to be for rental properties, this should not be the case as this data is to supposed to contain only properties for sale. Now to clean the price column and change its data type to float. We make use of a list comprehension and the regular expressions 're' module just as we used above to weed out the dirty data.

In [57]:
lag["Price"] = [float(re.findall("\d+", val.split("$")[1])[0]) * 775 \
                if "$" in val else float(re.findall("\d+", val)[0])\
                      for val in lag['Price']]

A brief explanation of the code above: The list comprehension loops through price column checks for values with Dollar signs, if they exist value is split into two, the dollar sign discarded then value is checked for other special characters,then the split portion is typecast to float and converted to Naira by multiplying by the exchange rate. The other part of the conditional handles the price rows with no dollar signs

If the code previusly used to check if the price column had any dirty data is run an Attribute Error is returned as the column no longer has any string datatypes. The conversion to float was also successful because all symbols were successfuly removed.

Next up, check for missing values

In [58]:
# check for missing values
print(lag.isna().any()) # to check if any columns have missing values
print(' ')
print('Sum of missing values per column:')
lag.isna().sum() # to get a sum of missing values per column

Description     False
Title           False
Location        False
Beds             True
Baths            True
Toilets          True
Is_new          False
Is_furnished    False
Is_serviced     False
Price           False
dtype: bool
 
Sum of missing values per column:


Description       0
Title             0
Location          0
Beds            503
Baths           603
Toilets         619
Is_new            0
Is_furnished      0
Is_serviced       0
Price             0
dtype: int64

Important to note that while the 'any' method lets you know if any column has null values the 'sum' method applied lets you add the number of missing values to give a total per column.

The above output shows that only 3 columns have missing or unknown values. Now to know what percentage those missing values constitute per column.

In [59]:
# to check percentage of missing values per column
print('Percentage of missing values per column:')
print((lag.isna().sum()/ len(lag) *100))

Percentage of missing values per column:
Description     0.000000
Title           0.000000
Location        0.000000
Beds            2.030436
Baths           2.434102
Toilets         2.498688
Is_new          0.000000
Is_furnished    0.000000
Is_serviced     0.000000
Price           0.000000
dtype: float64


The percentage of missing values in those 3 columns doesnt exceed 3% of each column so it is safe to conclude that dropping these columns will not have any adverse impact on the outcome of any predictive analysis to be carried out. So to drop the missing values use the 'dropna' method and set appropriate parameters.

In [60]:
# drop rows with missing values
lag.dropna(axis=0, inplace=True)

Next, check for duplicate data.Use the 'duplicated' method with the 'sum' method chained to it to get the number of missing values. Like below:

In [61]:
# check for duplicates
lag.duplicated().sum()

2150

A seen above there are 2150 duplicate rows in in the data.That data needs to be dropped.

In [62]:
# remove duplicates
lag.drop_duplicates(inplace=True)

Whitespace can cause all sorts of errors in analysis, so for each column that has string values, the whitespace must be removed.

In [63]:
# remove whitspace from string columns
for col in lag.columns:
        if lag[col].dtype == "object":
            lag[col] = lag[col].str.strip()

Some transformations need to be performed on the data, so reasonable insights can be extracted from the data.

In [65]:
# extract housetypes from the title column
lag['House_type'] = lag['Title'].apply(lambda x: x.split(' ')[2])

In [66]:
lag.House_type.value_counts()

HOUSE         16296
FLAT           2578
SALE           2087
FOR             641
APARTMENT       210
COMMERCIAL      116
LAND              7
SPACE             2
DETACHED          1
Name: House_type, dtype: int64

In [67]:
odd_house_types = lag[(lag['House_type'] == 'LAND') | \
                      (lag['House_type'] == 'SALE') |\
                        (lag['House_type'] == 'FOR') | \
                             (lag['House_type'] == 'COMMERCIAL') | \
                             (lag['House_type'] == 'SPACE') | \
                             (lag['House_type'] == 'DETACHED')
                             ].index

The other house types that are not House, apartment or flat are referring to land or some commercial property and those are not fit for the purpose of predicting residential property sale prices.

In [69]:
# drop odd house types
lag.drop(odd_house_types, axis=0,inplace=True)

In [70]:
lag['House_type'].value_counts()

HOUSE        16296
FLAT          2578
APARTMENT      210
Name: House_type, dtype: int64

In [80]:
# replace flat with just apartment
lag['House_type'] = lag['House_type'].str.replace('FLAT', 'APARTMENT')

In [83]:
# rename House_type column to Property_type
lag.rename(columns={'House_type':'Property_type'}, inplace=True)

In [84]:
# make values in column lower case
lag['Property_type'] = lag['Property_type'].str.lower()

Next, extract the unique locations or neighborhoods from the location's column and map to appropriate Local Government Areas.

In [120]:
# extract unique locations from the location column
lag['neighborhood'] = lag['Location'].apply(lambda x: x.split(' ')[-2])
lag['state'] = lag['Location'].apply(lambda x: x.split(' ')[-1])
# lag.sub_Location.value_counts()

Some properties are not in Lagos and so they must be removed.

In [97]:
# drop rows with data from locations not in Lagos
odd_locations = lag[(lag['state']=='Ogun') | (lag['state']=='Oyo')].index
lag.drop(odd_locations, axis=0, inplace=True)


In [121]:
# to get uninque locations
locations_lag = list(lag['neighborhood'].value_counts().index)

In [134]:
len(locations_lag)

38

There are 38 unique locations in the neighborhood column. Now to map these locations to the Local government areas in Lagos. A dictionary is used, with the neighborhood used as key and the Local Government Area as a value. This dictionary is then passed to the replace function.

In [138]:
lga_dict = {'Lekki':'Eti-Osa', 'Ajah':'Eti-Osa', 'Ikoyi':'Eti-Osa', 'Ikeja':'Ikeja',
            'Ojodu':'Ikeja', 'Island':'Eti-Osa','Alimosho':'Alimosho','Gbagada':'Kosofe','Ikosi':'Epe',
            'Surulere':'Surulere','Ikorodu':'Ikorodu','Ipaja':'Alimosho','Yaba':'Lagos Mainland','Igando':'Alimosho',
            'Isolo':'Oshodi-Isolo','Ogba':'Ifako-Ijaiye','Egba':'Alimosho','Maryland':'Kosofe','Igando':'Alimosho',
            'Ogudu':'Kosofe','Okota':'Oshodi-Isolo','Agege':'Agege','Sangotedo':'Eti-Osa','Shomolu':'Shomolu','Odofin':'Amuwo-Odofin',
            'Ilupeju':'Mushin','Ketu':'Kosofe','Idimu':'Alimosho','Iju':'Ifako-Ijaiye','Epe':'Epe','Oshodi':'Oshodi-Isolo','Ojota':'Kosofe',
            'Ejigbo':'Oshodi-Isolo','Bariga':'Shomolu','Apapa':'Apapa','Ojo':'Ojo','Badagry':'Badagry','Mushin':'Mushin','Orile':'Agege'}

lag = lag.replace({'neighborhood':lga_dict})

Three columns contain numbers representing, yes or no. These columns also need to be looked at

In [89]:
# replace 1 and 0 in is_new, is_furnished, is_serviced with yes or no
map_cols = {1:'yes', 0:'no'}
lag.replace({'Is_new':map_cols, 'Is_furnished':map_cols, 'Is_serviced':map_cols },inplace=True)

This replacement can be avoided by making adjustments to the scrapper when scrapping to return yes or no instead of 1 or 0.The scrapper is the 'house_scrapper.py' script. These values will still be encoded later during model building but for analysis purposes the replacement is necessary.

Finally some unnecessary columns need to be dropped before the dataframe is saved to a csv file.

In [142]:
# drop unnecessary columns 
lag.drop(['Description', 'Title', 'Location','state'], axis=1, inplace=True)
# rename sub_Location column
lag.rename(columns={'neighborhood':'Location'}, inplace=True)

Now one last look at the data before its is written to a new csv file titled 'cleaned_houses.csv' found in the data folder of the master branch.

In [144]:
lag.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19074 entries, 0 to 24772
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Beds           19074 non-null  float64
 1   Baths          19074 non-null  float64
 2   Toilets        19074 non-null  float64
 3   Is_new         19074 non-null  object 
 4   Is_furnished   19074 non-null  object 
 5   Is_serviced    19074 non-null  object 
 6   Price          19074 non-null  float64
 7   Property_type  19074 non-null  object 
 8   Location       19074 non-null  object 
dtypes: float64(4), object(5)
memory usage: 1.5+ MB


In [157]:
# write clean data to csv file
lag.to_csv("cleaned_houses.csv", index=False)
